#### 音素情報を用いて候補単語についてフィルタリングする

- 短い音素の単語
- 似ている音素がある単語

In [1]:
cd ../data/external/

/home/saho/project/wovle/data/external


In [86]:
!wc -l ./hybrid.htkdic

322238 ./hybrid.htkdic


In [87]:
!head ./hybrid.htkdic

<s>	[]	sp_S
</s>	[。]	sp_S
、+補助記号	[、]	sp_S
×+名詞	[×]	b_B a_I ts_I u_E
Δ+記号	[Δ]	d_B e_I r_I u_I t_I a_E
Θ+記号	[Θ]	sh_B i:_I t_I a_E
Σ+記号	[Σ]	sh_B i_I g_I u_I m_I a_E
Φ+記号	[Φ]	f_B a_I i_E
Ψ+記号	[Ψ]	p_B u_I s_I a_I i_E
Ω+名詞	[Ω]	o:_B m_I u_E


In [88]:
dic = pd.read_csv('./hybrid.htkdic', sep='\t', header=None, names=[0,1,2,3,4], low_memory=False)

In [89]:
dic.head()

,0,1,2,3,4
0,<s>,[],sp_S,NaN,NaN
1,</s>,[。],sp_S,NaN,NaN
2,、+補助記号,[、],sp_S,NaN,NaN
3,×+名詞,[×],b_B a_I ts_I u_E,NaN,NaN
4,Δ+記号,[Δ],d_B e_I r_I u_I t_I a_E,NaN,NaN


In [90]:
dic[dic[4].str.len() > 0].head()

,0,1,2,3,4
37,Ⅰ+名詞,@-0.0115,イチ,[Ⅰ],i_B ch_I i_E
38,Ⅰ+名詞,@-1.5846,イッ,[Ⅰ],i_B q_E
39,Ⅱ+名詞,@-0.0047,ニ,[Ⅱ],n_B i_E
40,Ⅱ+名詞,@-1.9718,ツー,[Ⅱ],ts_B u:_E
41,Ⅲ+名詞,@-0.0127,サン,[Ⅲ],s_B a_I N_E


In [91]:
buf = pd.DataFrame()
buf2 = pd.DataFrame()

In [92]:
buf['単語'] = dic[~(dic[4].str.len() > 0)][0]

In [93]:
buf['音素'] = dic[~(dic[4].str.len() > 0)][2]

In [94]:
buf2['単語'] = dic[dic[4].str.len() > 0][0]

In [95]:
buf2['音素'] = dic[dic[4].str.len() > 0][4]

In [118]:
onso = buf.append(buf2)

In [119]:
onso = onso.reset_index(drop=True)

In [120]:
onso.head()

,単語,音素
0,<s>,sp_S
1,</s>,sp_S
2,、+補助記号,sp_S
3,×+名詞,b_B a_I ts_I u_E
4,Δ+記号,d_B e_I r_I u_I t_I a_E


In [121]:
onso.shape

(322238, 2)

In [122]:
onso['音素'] = onso['音素'].apply(lambda x: [xx.replace('_B','').replace('_I','').replace('_E','').replace('_S','') for xx in x.split(' ')])

In [123]:
onso['isMistaking'] = False

In [124]:
onso.head()

,単語,音素,isMistaking
0,<s>,[sp],False
1,</s>,[sp],False
2,、+補助記号,[sp],False
3,×+名詞,"[b, a, ts, u]",False
4,Δ+記号,"[d, e, r, u, t, a]",False


### 3音素以下の単語にチェック

In [125]:
onso[('isMistaking')] = onso[('音素')].apply(lambda v: len(v) <= 3)

In [126]:
sum(onso['isMistaking'] == True)

8943

#### 音素列挙

In [127]:
onso.head()

,単語,音素,isMistaking
0,<s>,[sp],True
1,</s>,[sp],True
2,、+補助記号,[sp],True
3,×+名詞,"[b, a, ts, u]",False
4,Δ+記号,"[d, e, r, u, t, a]",False


In [128]:
onso_list = set()
for v in onso['音素'].values:
    for w in v:
        onso_list.add(w)

In [129]:
onso_list

{'N',
 'a',
 'a:',
 'b',
 'by',
 'ch',
 'd',
 'e',
 'e:',
 'f',
 'g',
 'gy',
 'h',
 'hy',
 'i',
 'i:',
 'j',
 'k',
 'ky',
 'm',
 'my',
 'n',
 'ny',
 'o',
 'o:',
 'p',
 'py',
 'q',
 'r',
 'ry',
 's',
 'sh',
 'sp',
 't',
 'ts',
 'u',
 'u:',
 'w',
 'y',
 'z'}

### 似ている音素で一致している組み合わせがあればチェック (なんか意味無さそうなので却下)

- \* と \*:
- s と sh
- p と py

In [131]:
def isSame(a, b):
    if a == b:
        True
    elif a+':' == b or a == b+':':
        True
    elif a == 's' and b == 'sh':
        True
    elif a == 'sh' and b == 's':
        True
    elif a == 'p' and b == 'py':
        True
    elif a == 'py' and b == 'p':
        True
    else:
        False

### フラッグ付きで保存

In [139]:
onso.head()

,単語,音素,isMistaking
0,<s>,[sp],True
1,</s>,[sp],True
2,、+補助記号,[sp],True
3,×+名詞,"[b, a, ts, u]",False
4,Δ+記号,"[d, e, r, u, t, a]",False


In [ ]:
onso_dic = onso[['単語','isMistaking']]

In [138]:
onso.groupby(['単語']).mean()

,isMistaking
単語,
$0,0.0
$1,0.0
$10,0.0
$100,0.0
$1000,0.0
$12,0.0
$15,0.0
$150,0.0
$2,0.0
